# Monopod on Skateboard
### Model a monopod to apply backfoot applied force on the skateboard


It's a skateboard model with monopod applying the back force on the board and a force (only perpendicular) applying a force on the front of the board. The results are shown in "frame" time and real time.

You can choose to initialize the model with an existing solved solution (essential).

In [1]:
# import the script that holds a bunch of the re-used functions
import skaterlib
from importlib import reload
reload(skaterlib);

import cloudpickle

with open('saved_models\\model_save_Neq60.pkl', mode='rb') as file:
    m_saved = cloudpickle.load(file)

In [24]:
# import libraries
import pickle as pkl
import sympy as sym
import numpy as np

sym.init_printing()
from IPython.display import display #for pretty printing

# DERIVE EOMS OF BOARD --------------------------------------------------------------------------------------------------------

# system parameters
g = sym.symbols('g')
mb = sym.symbols('m_{board}') # mass
lb = sym.symbols('l_{board}') # length
lbr = sym.symbols('l_{wheels}') # length to reaction forces
hb = sym.symbols('h_{board}') # height - board clearance
Inb = sym.symbols('In_{board}') # moment of intertia
rF_FF, rF_BF = sym.symbols(['r_{F_{FF}}','r_{F_{BF}}']) # distance of feet from COM. 

# generalized coordinates
x,y,thb = sym.symbols(['x','y','\\theta_{board}']) 
dx,dy,dthb = sym.symbols(['\dot{x}','\dot{y}','\dot{\\theta}_{board}']) 
ddx,ddy,ddthb = sym.symbols(['\ddot{x}','\ddot{y}','\ddot{\\theta}_{board}']) 

q = sym.Matrix([[x],[y],[thb]])
dq = sym.Matrix([[dx],[dy],[dthb]])
ddq = sym.Matrix([[ddx],[ddy],[ddthb]])

# forces
GRF_BW,GRF_FW = sym.symbols(['GRF_{BW}','GRF_{FW}']) # ground reaction forces
F_BFx_sb,F_BFy_sb = sym.symbols(['F_{BFx_{sb}}','F_{BFy_{sb}}']) # back foot applied forces
F_FFx_sb,F_FFy_sb = sym.symbols(['F_{FFx_{sb}}','F_{FFy_{sb}}']) # front foot applied forces

# Unpickle stored EOMs
infile = open('skateboard_EOMs','rb')
data = pkl.load(infile)
infile.close()

EOMs_board = data['EOMs_Fsb']

In [25]:
# Lambdify EOMs
from pyomo.environ import*
from pyomo.opt import SolverFactory
from pyomo.opt import SolverStatus, TerminationCondition

func_map = {'sin':sin, 'cos':cos} 

sym_list = [g,mb,lb,lbr,Inb,hb,
            x,y,thb,
            dx,dy,dthb,
            ddx,ddy,ddthb,
            rF_BF,rF_FF,
            F_BFx_sb,F_BFy_sb,F_FFx_sb,F_FFy_sb,
            GRF_BW,GRF_FW]
            
lambEOMx_board   = sym.lambdify(sym_list,EOMs_board[0],modules = [func_map])
lambEOMy_board   = sym.lambdify(sym_list,EOMs_board[1],modules = [func_map])
lambEOMth_board  = sym.lambdify(sym_list,EOMs_board[2],modules = [func_map])

In [26]:
# DERIVE EOMS OF MONOPOD --------------------------------------------------------------------------------------------------------

from IPython.display import display #for pretty printing

# create symbolic variables

# system parameters
g = sym.symbols('g')
mb,ml1,ml2 = sym.symbols(['m_{body}','m_{leg1}','m_{leg2}']) # mass
lb,ll1,ll2 = sym.symbols(['l_{body}','l_{leg1}','l_{leg2}']) # length
Inb,Inl1,Inl2 = sym.symbols(['I_{body}','I_{leg1}','I_{leg2}']) # moment of intertia

# generalized coordinates
x,y,thb,thl,r = sym.symbols(['x','y','\\theta_{body}','\\theta_{leg}','r']) 
dx,dy,dthb,dthl,dr = sym.symbols(['\dot{x}','\dot{y}','\dot{\\theta}_{body}','\dot{\\theta}_{leg}','\dot{r}']) 
ddx,ddy,ddthb,ddthl,ddr = sym.symbols(['\ddot{x}','\ddot{y}','\ddot{\\theta}_{body}','\ddot{\\theta}_{leg}','\ddot{r}']) 

q = sym.Matrix([[x],[y],[thb],[thl],[r]])
dq = sym.Matrix([[dx],[dy],[dthb],[dthl],[dr]])
ddq = sym.Matrix([[ddx],[ddy],[ddthb],[ddthl],[ddr]])

# forces
# total joint action = actuator + rebound, but that will be dealt with elsewhere
F,tau,GRFx,GRFy = sym.symbols(['F','\\tau','G_x','G_y']) 

# Unpickle stored EOMs
infile = open('monopod_EOMs','rb')
data = pkl.load(infile)
infile.close()

EOMs_monopod = data['EOMs']

In [27]:
# Lambdify EOMs of Monopod
from pyomo.environ import*
from pyomo.opt import SolverFactory
from pyomo.opt import SolverStatus, TerminationCondition

func_map = {'sin':sin, 'cos':cos} 

sym_list = [g,mb,ml1,ml2,lb,ll1,ll2,Inb,Inl1,Inl2,
            x,y,thb,thl,r,
            dx,dy,dthb,dthl,dr,
            ddx,ddy,ddthb,ddthl,ddr,
            F,tau,GRFx,GRFy]
            
lambEOMxb_monopod = sym.lambdify(sym_list,EOMs_monopod[0],modules = [func_map])
lambEOMyb_monopod = sym.lambdify(sym_list,EOMs_monopod[1],modules = [func_map])
lambEOMthb_monopod = sym.lambdify(sym_list,EOMs_monopod[2],modules = [func_map])
lambEOMthl_monopod = sym.lambdify(sym_list,EOMs_monopod[3],modules = [func_map])
lambEOMr_monopod = sym.lambdify(sym_list,EOMs_monopod[4],modules = [func_map])

In [28]:
# Unpickle foot position and velocity

xm,ym,thm,thl,r = sym.symbols(['x_{mono}','y_{mono}','\\theta_{mono}','\\theta_{link1}','r_{mono}']) # monopod in global frame
dxm,dym,dthm,dthl,dr = sym.symbols(['\dot{x}_{mono}','\dot{y}_{mono}','\dot{\\theta}_{mono}','\dot{\\theta}_{link1}','\dot{r}_{mono}'])
l1,l2 = sym.symbols(['l_{link1}','l_{link2}']) # lenghts of links
xb,yb,thb = sym.symbols(['x_{board}','y_{board}','\\theta_{board}']) # board in global frame
dxb,dyb,dthb = sym.symbols(['\dot{x}_{board}','\dot{y}_{board}','\dot{\\theta}_{board}'])

# Unpickle stored EOMs
infile = open('monopod_EOMs','rb')
data = pkl.load(infile)
infile.close()

pfoot = data['pfoot']
pfoot_board = data['pfoot_board']
vfootx_board = data['vfootx_board']

# Lambdify pfoot_board
func_map = {'sin':sin, 'cos':cos, 'atan':atan} 
sym_list = [l1, l2, xb, yb, thb, xm, ym, thm, thl, r]
sym_list2 = [l1, l2, xb, yb, thb, xm, ym, thm, thl, r, dxb, dyb, dthb, dxm, dym, dthm, dthl, dr]

lambfootx = sym.lambdify(sym_list,pfoot[0],modules = [func_map])
lambfooty = sym.lambdify(sym_list,pfoot[1],modules = [func_map])
lambfootx_board = sym.lambdify(sym_list,pfoot_board[0],modules = [func_map])
lambfooty_board = sym.lambdify(sym_list,pfoot_board[1],modules = [func_map])
lambvfootx_board = sym.lambdify(sym_list2,vfootx_board,modules = [func_map])

In [29]:
if 'm' in globals():
    del m # deletes the model
    
m = ConcreteModel()

# SETS-----------------------------------------------------------------------------------------------------------------------

N = 60
N1 = 16
N2 = 50
m.N = RangeSet(N)

DOFs = ['x','y','th','xb','yb','thb','thl','r'] # generalized coordinates
m.DOF = Set(initialize = DOFs) 

Fs = ['BF','FF'] # front foot, backfoot
m.Fs = Set(initialize = Fs) 

GRFs = ['BW','FW'] # front wheel, back wheel
m.GRFs = Set(initialize = GRFs) 

links = [('body',1),('leg',1),('leg',2)]
m.L = Set(dimen=2, initialize = links)

WDOFs = ['X','Y'] # absolute coordinates
m.WDOF = Set(initialize = WDOFs)

DOF_b = ['x_b','y_b'] # coordinates in skateboard frame
m.DOF_b = Set(initialize = DOF_b)

DOF_m = ['x_m','y_m'] # coordinates in monopod frame
m.DOF_m = Set(initialize = DOF_m)

bodies = ['board','Bfoot'] # coordinates in monopod frame
m.bodies = Set(initialize = bodies)

# PARAMETERS-----------------------------------------------------------------------------------------------------------------

# general
m.g = Param(initialize = 9.81) # gravity

# board
m.mbd = Param(initialize = 2.0) # mass of board
m.lbd = Param(initialize = 0.80) # length of board
m.lbrd = Param(initialize = 0.45) # length between wheels lb-(14.3+3)*2
m.hbd = Param(initialize = 0.09) # board clearance
m.etail = Param(initialize = -1.0) #coefficient of restitution of tail
m.Inbd = Param(initialize = 1/12*m.mbd*m.lbd**2) # moment of inertia about centre

BW_sb = m.mbd*m.g

# monopod
def get_m(n, lb, ln):
    if lb == 'body':
        return 5.0
    else: return 2.5
m.m = Param(m.L, initialize = get_m) # mass of links

def get_len(n, lb, ln):
    if lb == 'body':
        return 1.0
    else: return 0.5
m.len = Param(m.L, initialize = get_len) # length of links

def calculate_In(m, lb, ln): 
    l = (lb,ln)
    return 1/12*m.m[l]*m.len[l]**2 
m.In = Param(m.L, initialize = calculate_In) # moment of inertia

m_monopod = sum(m.m[l] for l in links)
BW = m_monopod*m.g

# VARIABLES -----------------------------------------------------------------------------------------------------------------

# system coordinates
m.q = Var(m.N, m.DOF) # position
m.dq = Var(m.N, m.DOF) # velocity
m.ddq = Var(m.N, m.DOF) # acceleration

# applied and ground reaction forces and torques
# sign set for positive and negative components
signs = ['ps','ng'] 
m.sgn = Set(initialize = signs)

m.Fb_b = Var(m.N, m.Fs, m.DOF_b, m.sgn, bounds = (0.0,50.0)) # Forces from front and back feet, board frame
m.Fb_b_total = Var(m.N, m.Fs, m.DOF_b)
m.GRF = Var(m.N, m.GRFs, bounds = (0.0, None)) # Ground reaction force on back wheel
m.GRFbf = Var(m.N, m.WDOF) # Board reaction force on foot, global frame
m.tau_a = Var(m.N, bounds = (-10,10)) # actuator torque at hip
m.F_a = Var(m.N, bounds = (-50,50)) # actuator force at knee
m.F_r = Var(m.N, m.sgn, bounds = (0.0,None)) # rebound force (acts parallel to the leg)

# AUXILIARY VARIABLES

# skateboard
m.ptail = Var(m.N, m.WDOF) # back of board
m.pnose = Var(m.N, m.WDOF) # front of board
m.pwheel = Var(m.N, m.GRFs, m.WDOF) # position of wheels
m.vtail = Var(m.N, m.WDOF) # velocity of back of board

def def_ptail(m,n,dof):
    if dof == 'X':
        return m.ptail[n, 'X'] == m.q[n,'x'] - 0.5*m.lbd*cos(m.q[n,'th'])
    if dof == 'Y':
        return m.ptail[n, 'Y'] == m.q[n,'y'] - 0.5*m.lbd*sin(m.q[n,'th']) 
    else:
        return Constraint.Skip
m.def_ptail = Constraint(m.N, m.WDOF,rule = def_ptail)

def def_pnose(m,n,dof):
    if dof == 'X':
        return m.pnose[n, 'X'] == m.q[n,'x'] + 0.5*m.lbd*cos(m.q[n,'th'])
    if dof == 'Y':
        return m.pnose[n, 'Y'] == m.q[n,'y'] + 0.5*m.lbd*sin(m.q[n,'th']) 
    else:
        return Constraint.Skip
m.def_pnose = Constraint(m.N, m.WDOF,rule = def_pnose)

def def_pwheels(m,n,wheel,dof):
    if wheel == 'BW':
        if dof == 'X':
            return m.pwheel[n,'BW','X'] == m.q[n,'x']-0.5*m.lbrd*cos(m.q[n,'th'])+m.hbd*sin(m.q[n,'th'])
        if dof == 'Y':
            return m.pwheel[n,'BW','Y'] == m.q[n,'y']-0.5*m.lbrd*sin(m.q[n,'th'])-m.hbd*cos(m.q[n,'th'])
        else:
            return Constraint.Skip
    if wheel == "FW":
        if dof == 'X':
            return m.pwheel[n,'FW','X'] == m.q[n,'x']+0.5*m.lbrd*cos(m.q[n,'th'])+m.hbd*sin(m.q[n,'th'])
        if dof == 'Y':
            return m.pwheel[n,'FW','Y'] == m.q[n,'y']+0.5*m.lbrd*sin(m.q[n,'th'])-m.hbd*cos(m.q[n,'th'])
        else:
            return Constraint.Skip
    else:
        return Constraint.Skip
m.def_pwheels = Constraint(m.N, m.GRFs, m.WDOF,rule = def_pwheels)

def def_vtail(m,n,dof):
    if dof == 'Y':
        return m.vtail[n, 'Y'] == m.dq[n,'y']-m.dq[n,'th']*0.5*m.lbd*cos(m.q[n,'th'])
    else:
        return Constraint.Skip
m.def_vtail = Constraint(m.N, m.WDOF,rule = def_vtail)

# skateboard - bound variables
for n in range(1,N+1):
    m.pnose[n,'Y'].setlb(0.0)
    m.ptail[n,'Y'].setlb(0.0)
    m.pwheel[n,'BW','Y'].setlb(0.0)
    m.pwheel[n,'FW','Y'].setlb(0.0)
    m.q[n,'th'].setub(np.pi/2)
    m.q[n,'th'].setlb(-np.pi/2)
    
# monopod - bound variables
for n in range(1,N+1):  
    m.q[n,'thb'].setlb(-np.pi/2)
    m.q[n,'thb'].setub(np.pi/2)
    m.q[n,'thl'].setlb(-np.pi/2)
    m.q[n,'thl'].setub(np.pi/2)
    #m.GRFbf[n,'Y'].setlb(0.0)

# monopod 
m.pfoot = Var(m.N, m.WDOF) # foot position
m.pfoot_b = Var(m.N, m.DOF_b) # foot position in skateboard frame
m.vfoot_b = Var(m.N, m.DOF_b, m.sgn, bounds = (0.0,None)) # foot velocity
m.vfoot_b2 = Var(m.N, m.DOF_b,) # foot velocity
m.friction_cone = Var(m.N, m.Fs, bounds = (0.0,None))

for n in range(1,N+1):  
    m.pfoot[n,'Y'].setlb(0.0)
    m.pfoot_b[n,'y_b'].setlb(0.0)
    m.pfoot_b[n,'x_b'].setub(0.0)
    m.pfoot_b[n,'x_b'].setlb(-0.5*m.lbd)

def def_pfoot(m,n,dof):
    # world frame
    var_list = [m.len[('leg',1)],m.len[('leg',2)],
                m.q[n,'x'], m.q[n,'y'], m.q[n,'th'], m.q[n,'xb'], m.q[n,'yb'], m.q[n,'thb'], m.q[n,'thl'], m.q[n,'r']]
    if dof == 'X':
        return m.pfoot[n,'X'] == lambfootx(*var_list)
    if dof == 'Y':
        return m.pfoot[n,'Y'] == lambfooty(*var_list)
    else:
        return Constraint.Skip
m.def_pfoot = Constraint(m.N, m.WDOF, rule = def_pfoot)    
    
def def_pfoot_b(m,n,dof):
    # world frame
    var_list = [m.len[('leg',1)],m.len[('leg',2)],
                m.q[n,'x'], m.q[n,'y'], m.q[n,'th'], m.q[n,'xb'], m.q[n,'yb'], m.q[n,'thb'], m.q[n,'thl'], m.q[n,'r']]
    if dof == 'x_b':
        return m.pfoot_b[n,'x_b'] == lambfootx_board(*var_list)
    if dof == 'y_b':
        return m.pfoot_b[n,'y_b'] == lambfooty_board(*var_list)
    else:
        return Constraint.Skip
m.def_pfoot_b = Constraint(m.N, m.DOF_b, rule = def_pfoot_b)      

def def_vfoot_b(m,n,dof):
    # board frame
    var_list = [m.len[('leg',1)],m.len[('leg',2)],
                m.q[n,'x'], m.q[n,'y'], m.q[n,'th'], m.q[n,'xb'], m.q[n,'yb'], m.q[n,'thb'], m.q[n,'thl'], m.q[n,'r'],
                m.dq[n,'x'], m.dq[n,'y'], m.dq[n,'th'], m.dq[n,'xb'], m.dq[n,'yb'], m.dq[n,'thb'], m.dq[n,'thl'], m.dq[n,'r']]
    if dof == 'x_b':
        return m.vfoot_b[n,'x_b','ps'] - m.vfoot_b[n,'x_b','ng'] == lambvfootx_board(*var_list)
    else:
        return Constraint.Skip
m.def_vfoot_b = Constraint(m.N, m.DOF_b, rule = def_vfoot_b)

def def_vfoot_b2(m,n,dof):
    # board frame
    var_list = [m.len[('leg',1)],m.len[('leg',2)],
                m.q[n,'x'], m.q[n,'y'], m.q[n,'th'], m.q[n,'xb'], m.q[n,'yb'], m.q[n,'thb'], m.q[n,'thl'], m.q[n,'r'],
                m.dq[n,'x'], m.dq[n,'y'], m.dq[n,'th'], m.dq[n,'xb'], m.dq[n,'yb'], m.dq[n,'thb'], m.dq[n,'thl'], m.dq[n,'r']]
    if dof == 'x_b':
        return m.vfoot_b2[n,'x_b'] == lambvfootx_board(*var_list)
    else:
        return Constraint.Skip
m.def_vfoot_b2 = Constraint(m.N, m.DOF_b, rule = def_vfoot_b2)


def def_Fb_b_total(m,n,fs,dof):
    if fs == 'BF':
        if dof == 'x_b':
            return m.Fb_b_total[n,'BF','x_b'] == m.Fb_b[n,'BF','x_b','ps']-m.Fb_b[n,'BF','x_b','ng']
        if dof == 'y_b':
            return m.Fb_b_total[n,'BF','y_b'] == m.Fb_b[n,'BF','y_b','ps']
        else:
            return Constraint.Skip
    if fs == 'FF':
        if dof == 'x_b':
            return m.Fb_b_total[n,'FF','x_b'] == m.Fb_b[n,'FF','x_b','ps']-m.Fb_b[n,'FF','x_b','ng']
        if dof == 'y_b':
            return m.Fb_b_total[n,'FF','y_b'] == m.Fb_b[n,'FF','y_b','ps']
    else:
        return Constraint.Skip
m.def_Fb_b_total = Constraint(m.N, m.Fs, m.DOF_b, rule = def_Fb_b_total) 

# Constraints for foot and board contact --------------------------------------------------------------------------------------

m.rF = Var(m.N, m.Fs) # distance to applied force

def def_rF(m,n): # contact point on board
    return m.rF[n,'BF'] == -m.pfoot_b[n,'x_b']
m.def_rF = Constraint(m.N, rule = def_rF)

In [30]:
#m.Fb_b_total.pprint()

In [31]:
# TIME AND INTEGRATION --------------------------------------------------------------------------------------------------------

# variable timestep
hm  = 0.01 #master timestep
m.h = Var(m.N, bounds = (0.8,1.2))

# Integration constraints 
def BwEuler_p(m,n,dof): # for positions
    if n > 1:
        return m.q[n,dof] == m.q[n-1,dof] + hm*m.h[n]*m.dq[n,dof]
    else:
        return Constraint.Skip 
m.integrate_p = Constraint(m.N, m.DOF, rule = BwEuler_p)

def BwEuler_v(m,n,dof): # for velocities
    if n > 1:
        if n==N1 and dof == 'y':
            return m.dq[n,'y'] == m.etail*m.vtail[n-1,'Y'] + m.dq[n-1,'y'] #switching constraint at impact of tail and ground
        else:    
            return m.dq[n,dof] == m.dq[n-1,dof] + hm*m.h[n]*m.ddq[n-1,dof]
    else:
        return Constraint.Skip 
m.integrate_v = Constraint(m.N, m.DOF, rule = BwEuler_v)

m.h[1].fix(0.0)

In [32]:
# AUXILIARY TO RESOLVE FOOT AND BOARD FORCES
# The back and front foot applied forces on the board, and the applied force of the monopod are in their own frames
# Need to transform them into the global frame.

m.F = Var(m.N, m.bodies,m.Fs,m.WDOF) # Forces, global frame
m.thA = Var(m.N) # absolute angle of monopod

def def_thA(m,n):
    return m.thA[n] == m.q[n,'thb'] + m.q[n,'thl']
m.def_thA = Constraint(m.N, rule = def_thA)

def def_F(m,n,body,fs,dof):
    if fs == 'BF':
        if body == 'Bfoot':
            if dof == 'X':
                return m.F[n,'Bfoot','BF','X'] == m.GRFbf[n,'X'] #+ m.F_a[n]*sin(m.thA[n])
            if dof == 'Y':
                return m.F[n,'Bfoot','BF','Y'] == m.GRFbf[n,'Y'] #- m.F_a[n]*cos(m.thA[n])
            else:
                return Constraint.Skip
        if body == 'board':
            if dof == 'X':
                return m.F[n,'board','BF','X'] == m.Fb_b_total[n,'BF','x_b']*cos(m.q[n,'th']) + m.Fb_b_total[n,'BF','y_b']*sin(m.q[n,'th'])
            if dof == 'Y':
                return m.F[n,'board','BF','Y'] == m.Fb_b_total[n,'BF','x_b']*sin(m.q[n,'th']) - m.Fb_b_total[n,'BF','y_b']*cos(m.q[n,'th'])
            else:
                return Constraint.Skip
    else:
        return Constraint.Skip
m.def_F = Constraint(m.N, m.bodies, m.Fs, m.WDOF, rule = def_F)

def def_contact_forces(m,n,fs,dof): # resolve into X and Y - global frame
    if fs == 'BF':
        if dof == 'X':
            return m.F[n,'Bfoot','BF','X'] == -m.F[n,'board','BF','X']
        if dof == 'Y':
            return m.F[n,'Bfoot','BF','Y'] == -m.F[n,'board','BF','Y']
        else:
            return Constraint.Skip
    else:
        return Constraint.Skip
m.def_contact_forces = Constraint(m.N, m.Fs, m.WDOF, rule = def_contact_forces)

In [33]:
#m.F_a.pprint()

In [34]:
# -----------------------------------------------------------------------------------------------------------------------------
# Contact Forces
# -----------------------------------------------------------------------------------------------------------------------------
# paramters
m.mu = Param(initialize = 1) # friction coefficient

ground_constraints = ['contact_BW','contact_FW']
m.ground_constraints = Set(initialize = ground_constraints) # set for indexing ground-related penalties
m.ground_penalty = Var(m.N, m.ground_constraints, bounds = (0.0,None))

# contact_constraints = ['contact_Fy','contact_foot_Fy', 'friction_BF', 'friction_FF','slip_x_b_ps','slip_x_b_ng','foot_velocity']
contact_constraints = ['contact_Fy', 'friction_BF', 'friction_FF','slip_x_b_ps','slip_x_b_ng','foot_velocity']
m.contact_constraints = Set(initialize = contact_constraints) # set for indexing ground-related penalties
m.contact_penalty = Var(m.N, m.contact_constraints, bounds = (0.0,None))

# ground contact complemetarity  
def ground_contact(m,n,gc):
    if n < N:
        if gc == 'contact_FW':
            return m.ground_penalty[n,'contact_FW'] == m.pwheel[n+1,'FW','Y']*m.GRF[n,'FW'] 
        if gc == 'contact_BW':
            return m.ground_penalty[n,'contact_BW'] == m.pwheel[n+1,'BW','Y']*m.GRF[n,'BW']
        else:
            return Constraint.Skip
    else:
        return Constraint.Skip
m.ground_contact = Constraint(m.N, m.ground_constraints, rule = ground_contact)

# foot on board contact complemetarity  
def foot_board_contact(m,n,cc):
    if n < N:
        if cc == 'contact_Fy':
            return m.contact_penalty[n,'contact_Fy'] == m.pfoot_b[n+1,'y_b']*m.Fb_b[n,'BF','y_b','ps']
#         if cc == 'contact_foot_Fy':
#             return m.contact_penalty[n,'contact_foot_Fy'] == m.pfoot_b[n+1,'y_b']*m.GRFbf[n,'Y']
        else:
            return Constraint.Skip
    else:
        return Constraint.Skip
m.foot_board_contact = Constraint(m.N, m.contact_constraints, rule = foot_board_contact)

# Friction
def def_friction_cone(m,n,fs):
    if fs == 'BF':
        return m.friction_cone[n,'BF'] == m.mu*m.Fb_b[n,'BF','y_b','ps'] - (m.Fb_b[n,'BF','x_b','ps']+m.Fb_b[n,'BF','x_b','ng'])
    else:
        return Constraint.Skip
m.def_friction_cone = Constraint(m.N, m.Fs, rule = def_friction_cone)

def board_friction(m,n,fs): # the foot can't move if not over max static friction, or no normal force.
    if fs == 'BF':
        return m.contact_penalty[n,'friction_BF'] == (m.vfoot_b[n,'x_b','ps']+m.vfoot_b[n,'x_b','ng'])*m.friction_cone[n,'BF']
    else:
        return Constraint.Skip
m.board_friction = Constraint(m.N, m.Fs, rule = board_friction)

# slipping
def board_slip_ps(m,n): # the friction force must be applied in opposite direction to velocity of foot.
    return m.contact_penalty[n,'slip_x_b_ps'] == m.vfoot_b[n,'x_b','ps']*m.Fb_b[n,'BF','x_b','ng']
m.board_slip_ps = Constraint(m.N, rule = board_slip_ps)

def board_slip_ng(m,n): # the friction force must be applied in opposite direction to velocity of foot.
    return m.contact_penalty[n,'slip_x_b_ng'] == m.vfoot_b[n,'x_b','ng']*m.Fb_b[n,'BF','x_b','ps']
m.board_slip_ng = Constraint(m.N, rule = board_slip_ng)

# foot velocity - only vf+ or vf- must be non-zero, not both.
def vfoot_b_penalty(m,n):
    return m.contact_penalty[n,'foot_velocity'] == m.vfoot_b[n,'x_b','ps']*m.vfoot_b[n,'x_b','ng']
m.vfoot_b_penalty = Constraint(m.N, rule = vfoot_b_penalty)

for grf in GRFs:
    m.GRF[N, grf].fix(0.0)
for fs in Fs:
    for dof in DOF_b:
        for sgn in signs:
            m.Fb_b[N, fs, dof, sgn].fix(0.0)
for gnd in ground_constraints:
    m.ground_penalty[N, gnd].fix(0.0)
for cc in contact_constraints:
    m.contact_penalty[N, cc].fix(0.0)


In [35]:
# # -----------------------------------------------------------------------------------------------------------------------------
# # Contact Forces
# # -----------------------------------------------------------------------------------------------------------------------------
# # paramters
# m.mu = Param(initialize = 1) # friction coefficient

# ground_constraints = ['contact_BW','contact_FW']
# m.ground_constraints = Set(initialize = ground_constraints) # set for indexing ground-related penalties
# m.ground_penalty = Var(m.N, m.ground_constraints, bounds = (0.0,None))

# contact_constraints = ['contact_Fy', 'friction_BF', 'friction_FF','slip_x_b_ps','slip_x_b_ng','foot_velocity']
# m.contact_constraints = Set(initialize = contact_constraints) # set for indexing ground-related penalties
# m.contact_penalty = Var(m.N, m.contact_constraints, bounds = (0.0,None))

# m.slack = Var(m.N, bounds = (0.0, None)) # slack variable

# def def_slack(m,n,sgn):
#     if sgn == 'ps':
#         return m.slack[n]+m.vfoot_b2[n,'x_b'] >= 0
#     if sgn == 'ng':
#         return m.slack[n]-m.vfoot_b2[n,'x_b'] >= 0
#     else:
#         return Constraint.Skip
# m.def_slack = Constraint(m.N, m.sgn, rule = def_slack)
    
# # ground contact complemetarity  
# def ground_contact(m,n,gc):
#     if n < N:
#         if gc == 'contact_FW':
#             return m.ground_penalty[n,'contact_FW'] == m.pwheel[n+1,'FW','Y']*m.GRF[n,'FW'] 
#         if gc == 'contact_BW':
#             return m.ground_penalty[n,'contact_BW'] == m.pwheel[n+1,'BW','Y']*m.GRF[n,'BW']
#         else:
#             return Constraint.Skip
#     else:
#         return Constraint.Skip
# m.ground_contact = Constraint(m.N, m.ground_constraints, rule = ground_contact)

# # foot on board contact complemetarity  
# def foot_board_contact(m,n,cc):
#     if n < N:
#         if cc == 'contact_Fy':
#             return m.contact_penalty[n,'contact_Fy'] == m.pfoot_b[n+1,'y_b']*m.Fb_b[n,'BF','y_b','ps']
#         else:
#             return Constraint.Skip
#     else:
#         return Constraint.Skip
# m.foot_board_contact = Constraint(m.N, m.contact_constraints, rule = foot_board_contact)

# # Friction
# def def_friction_cone(m,n,fs):
#     if fs == 'BF':
#         return m.friction_cone[n,'BF'] == m.mu*m.Fb_b[n,'BF','y_b','ps'] - (m.Fb_b[n,'BF','x_b','ps']+m.Fb_b[n,'BF','x_b','ng'])
#     else:
#         return Constraint.Skip
# m.def_friction_cone = Constraint(m.N, m.Fs, rule = def_friction_cone)

# def board_friction(m,n,fs): # the foot can't move if not over max static friction, or no normal force.   
#     return m.contact_penalty[n,'friction_BF'] == m.slack[n]*m.friction_cone[n,'BF']
# m.board_friction = Constraint(m.N, m.Fs, rule = board_friction)

# # slipping
# def board_slip_ps(m,n): # the friction force must be applied in opposite direction to velocity of foot.
#     return m.contact_penalty[n,'slip_x_b_ps'] == (m.slack[n] + m.vfoot_b2[n,'x_b'])*m.Fb_b[n,'BF','x_b','ng']
# m.board_slip_ps = Constraint(m.N, rule = board_slip_ps)

# def board_slip_ng(m,n): # the friction force must be applied in opposite direction to velocity of foot.
#     return m.contact_penalty[n,'slip_x_b_ng'] == (m.slack[n] - m.vfoot_b2[n,'x_b'])*m.Fb_b[n,'BF','x_b','ps']
# m.board_slip_ng = Constraint(m.N, rule = board_slip_ng)

# for grf in GRFs:
#     m.GRF[N, grf].fix(0.0)
# for fs in Fs:
#     for dof in DOF_b:
#         for sgn in signs:
#             m.Fb_b[N, fs, dof, sgn].fix(0.0)
# for gnd in ground_constraints:
#     m.ground_penalty[N, gnd].fix(0.0)
# for cc in contact_constraints:
#     m.contact_penalty[N, cc].fix(0.0)


In [36]:
#m.contact_penalty.pprint()
#m.pprint()

In [37]:
# Reaction force at knee

# sets
joints = ['knee']
m.J = Set(initialize = joints)

joint_constraints = ['up','lo'] # set of joint penalties
m.joint_constraints = Set(initialize = joint_constraints)

knee_bound = [0.0,0.5]
m.knee_bound = Param(m.joint_constraints, initialize = {'up':knee_bound[1],'lo':knee_bound[0]})

# we can bound the joint coordinates directly
for n in range(1,N+1):
    m.q[n,'r'].setlb(knee_bound[0])
    m.q[n,'r'].setub(knee_bound[1])
    
m.joint_penalty = Var(m.N, m.J, m.joint_constraints, bounds = (0.0,None))

def knee_limits(m,n,jc):
    if n < N:
        if jc == 'up':
            # NEXT distance
            return m.joint_penalty[n,'knee',jc] == (m.knee_bound['up'] - m.q[n+1,'r'])*m.F_r[n,'ng']
        else: # jc =='low'
            return m.joint_penalty[n,'knee',jc] == (m.q[n+1,'r'] - m.knee_bound['lo'])*m.F_r[n,'ps']
    else:
        return Constraint.Skip
m.knee_limits = Constraint(m.N, m.joint_constraints, rule = knee_limits)

# bound contact forces at last node
for sgn in signs:
    m.F_r[N,sgn].value = 0
    m.F_r[N,sgn].fixed = True
    
for jc in joint_constraints:
    for j in joints:
        m.joint_penalty[N,j,jc].fix(0.0)

In [38]:
# -----------------------------------------------------------------------------------------------------------------------------
# EOMs 
# -----------------------------------------------------------------------------------------------------------------------------

# EOMs of skateboard
def EOMx_board(m,n):
    BFx_in = BW*m.Fb_b_total[n,'BF','x_b']
    BFy_in = BW*m.Fb_b_total[n,'BF','y_b']
    FFx_in = BW*m.Fb_b_total[n,'FF','x_b']
    FFy_in = BW*m.Fb_b_total[n,'FF','y_b']
    GBW_in = BW*m.GRF[n,'BW']
    GFW_in = BW*m.GRF[n,'FW']
    
    # list the model versions of all quantities in the same order as sym_list
    var_list = [m.g, m.mbd, m.lbd, m.lbrd, m.Inbd, m.hbd, m.q[n,'x'], m.q[n,'y'], m.q[n,'th'],
               m.dq[n,'x'], m.dq[n,'y'], m.dq[n,'th'], m.ddq[n,'x'], m.ddq[n,'y'], m.ddq[n,'th'],
               m.rF[n,'BF'], m.rF[n,'FF'],
               BFx_in, BFy_in, FFx_in, FFy_in, 
               GBW_in, GFW_in]
    return lambEOMx_board(*var_list) == 0
m.EOMx_board = Constraint(m.N, rule = EOMx_board)

def EOMy_board(m,n):
    BFx_in = BW*m.Fb_b_total[n,'BF','x_b']
    BFy_in = BW*m.Fb_b_total[n,'BF','y_b']
    FFx_in = BW*m.Fb_b_total[n,'FF','x_b']
    FFy_in = BW*m.Fb_b_total[n,'FF','y_b']
    GBW_in = BW*m.GRF[n,'BW']
    GFW_in = BW*m.GRF[n,'FW']
    
    # list the model versions of all quantities in the same order as sym_list
    var_list = [m.g, m.mbd, m.lbd, m.lbrd, m.Inbd, m.hbd, m.q[n,'x'], m.q[n,'y'], m.q[n,'th'],
               m.dq[n,'x'], m.dq[n,'y'], m.dq[n,'th'], m.ddq[n,'x'], m.ddq[n,'y'], m.ddq[n,'th'],
               m.rF[n,'BF'], m.rF[n,'FF'],
               BFx_in, BFy_in, FFx_in, FFy_in, 
               GBW_in, GFW_in]
    return lambEOMy_board(*var_list) == 0
m.EOMy_board = Constraint(m.N, rule = EOMy_board)

def EOMth_board(m,n):
    BFx_in = BW*m.Fb_b_total[n,'BF','x_b']
    BFy_in = BW*m.Fb_b_total[n,'BF','y_b']
    FFx_in = BW*m.Fb_b_total[n,'FF','x_b']
    FFy_in = BW*m.Fb_b_total[n,'FF','y_b']
    GBW_in = BW*m.GRF[n,'BW']
    GFW_in = BW*m.GRF[n,'FW']
    
    # list the model versions of all quantities in the same order as sym_list
    var_list = [m.g, m.mbd, m.lbd, m.lbrd, m.Inbd, m.hbd, m.q[n,'x'], m.q[n,'y'], m.q[n,'th'],
               m.dq[n,'x'], m.dq[n,'y'], m.dq[n,'th'], m.ddq[n,'x'], m.ddq[n,'y'], m.ddq[n,'th'],
               m.rF[n,'BF'], m.rF[n,'FF'],
               BFx_in, BFy_in, FFx_in, FFy_in, 
               GBW_in, GFW_in]
    return lambEOMth_board(*var_list) == 0
m.EOMth_board = Constraint(m.N, rule = EOMth_board)

# EOMs of monopod
def EOMxb_monopod(m,n):
    GRFx_in = BW*m.GRFbf[n,'X']
    GRFy_in = BW*m.GRFbf[n,'Y']
    F_a_in = BW*(m.F_a[n] + m.F_r[n,'ps'] - m.F_r[n,'ng'])
    tau_a_in = BW*m.tau_a[n]
    
    # list the model versions of all quantities in the same order as sym_list
    var_list = [m.g,m.m[('body',1)],m.m[('leg',1)],m.m[('leg',2)],
                m.len[('body',1)],m.len[('leg',1)],m.len[('leg',2)],
                m.In[('body',1)],m.In[('leg',1)],m.In[('leg',2)],
                m.q[n,'xb'],m.q[n,'yb'],m.q[n,'thb'],m.q[n,'thl'],m.q[n,'r'],
                m.dq[n,'xb'],m.dq[n,'yb'],m.dq[n,'thb'],m.dq[n,'thl'],m.dq[n,'r'],
                m.ddq[n,'xb'],m.ddq[n,'yb'],m.ddq[n,'thb'],m.ddq[n,'thl'],m.ddq[n,'r'],
                F_a_in, tau_a_in, GRFx_in, GRFy_in]
    return lambEOMxb_monopod(*var_list) == 0
m.EOMxb_monopod = Constraint(m.N, rule = EOMxb_monopod)

def EOMyb_monopod(m,n):
    GRFx_in = BW*m.GRFbf[n,'X']
    GRFy_in = BW*m.GRFbf[n,'Y']
    F_a_in = BW*(m.F_a[n] + m.F_r[n,'ps'] - m.F_r[n,'ng'])
    tau_a_in = BW*m.tau_a[n]
    
    # list the model versions of all quantities in the same order as sym_list
    var_list = [m.g,m.m[('body',1)],m.m[('leg',1)],m.m[('leg',2)],
                m.len[('body',1)],m.len[('leg',1)],m.len[('leg',2)],
                m.In[('body',1)],m.In[('leg',1)],m.In[('leg',2)],
                m.q[n,'xb'],m.q[n,'yb'],m.q[n,'thb'],m.q[n,'thl'],m.q[n,'r'],
                m.dq[n,'xb'],m.dq[n,'yb'],m.dq[n,'thb'],m.dq[n,'thl'],m.dq[n,'r'],
                m.ddq[n,'xb'],m.ddq[n,'yb'],m.ddq[n,'thb'],m.ddq[n,'thl'],m.ddq[n,'r'],
                F_a_in, tau_a_in, GRFx_in, GRFy_in]
    return lambEOMyb_monopod(*var_list) == 0
m.EOMyb_monopod = Constraint(m.N, rule = EOMyb_monopod)

def EOMthb_monopod(m,n):
    GRFx_in = BW*m.GRFbf[n,'X']
    GRFy_in = BW*m.GRFbf[n,'Y']
    F_a_in = BW*(m.F_a[n] + m.F_r[n,'ps'] - m.F_r[n,'ng'])
    tau_a_in = BW*m.tau_a[n]
    
    # list the model versions of all quantities in the same order as sym_list
    var_list = [m.g,m.m[('body',1)],m.m[('leg',1)],m.m[('leg',2)],
                m.len[('body',1)],m.len[('leg',1)],m.len[('leg',2)],
                m.In[('body',1)],m.In[('leg',1)],m.In[('leg',2)],
                m.q[n,'xb'],m.q[n,'yb'],m.q[n,'thb'],m.q[n,'thl'],m.q[n,'r'],
                m.dq[n,'xb'],m.dq[n,'yb'],m.dq[n,'thb'],m.dq[n,'thl'],m.dq[n,'r'],
                m.ddq[n,'xb'],m.ddq[n,'yb'],m.ddq[n,'thb'],m.ddq[n,'thl'],m.ddq[n,'r'],
                F_a_in, tau_a_in, GRFx_in, GRFy_in]
    return lambEOMthb_monopod(*var_list) == 0
m.EOMthb_monopod = Constraint(m.N, rule = EOMthb_monopod)

def EOMthl_monopod(m,n):
    GRFx_in = BW*m.GRFbf[n,'X']
    GRFy_in = BW*m.GRFbf[n,'Y']
    F_a_in = BW*(m.F_a[n] + m.F_r[n,'ps'] - m.F_r[n,'ng'])
    tau_a_in = BW*m.tau_a[n]
    
    # list the model versions of all quantities in the same order as sym_list
    var_list = [m.g,m.m[('body',1)],m.m[('leg',1)],m.m[('leg',2)],
                m.len[('body',1)],m.len[('leg',1)],m.len[('leg',2)],
                m.In[('body',1)],m.In[('leg',1)],m.In[('leg',2)],
                m.q[n,'xb'],m.q[n,'yb'],m.q[n,'thb'],m.q[n,'thl'],m.q[n,'r'],
                m.dq[n,'xb'],m.dq[n,'yb'],m.dq[n,'thb'],m.dq[n,'thl'],m.dq[n,'r'],
                m.ddq[n,'xb'],m.ddq[n,'yb'],m.ddq[n,'thb'],m.ddq[n,'thl'],m.ddq[n,'r'],
                F_a_in, tau_a_in, GRFx_in, GRFy_in]
    return lambEOMthl_monopod(*var_list) == 0
m.EOMthl_monopod = Constraint(m.N, rule = EOMthl_monopod)

def EOMr_monopod(m,n):
    GRFx_in = BW*m.GRFbf[n,'X']
    GRFy_in = BW*m.GRFbf[n,'Y']
    F_a_in = BW*(m.F_a[n] + m.F_r[n,'ps'] - m.F_r[n,'ng'])
    tau_a_in = BW*m.tau_a[n]
    
    # list the model versions of all quantities in the same order as sym_list
    var_list = [m.g,m.m[('body',1)],m.m[('leg',1)],m.m[('leg',2)],
                m.len[('body',1)],m.len[('leg',1)],m.len[('leg',2)],
                m.In[('body',1)],m.In[('leg',1)],m.In[('leg',2)],
                m.q[n,'xb'],m.q[n,'yb'],m.q[n,'thb'],m.q[n,'thl'],m.q[n,'r'],
                m.dq[n,'xb'],m.dq[n,'yb'],m.dq[n,'thb'],m.dq[n,'thl'],m.dq[n,'r'],
                m.ddq[n,'xb'],m.ddq[n,'yb'],m.ddq[n,'thb'],m.ddq[n,'thl'],m.ddq[n,'r'],
                F_a_in, tau_a_in, GRFx_in, GRFy_in]
    return lambEOMr_monopod(*var_list) == 0
m.EOMr_monopod = Constraint(m.N, rule = EOMr_monopod)

In [39]:
# COST FUNCTION ---------------------------------------------------------------------------------------------------------------

# minimum force and penalties
def CostFun(m):
    T = sum(m.h[n] for n in range(1,N+1))
    penalty_sum = 0
    force_sum = 0
    torque_sum = 0
    for n in range(1,N+1):
        for gc in ground_constraints:
            penalty_sum += m.ground_penalty[n,gc]
        for cc in contact_constraints:
            penalty_sum += m.contact_penalty[n,cc]
        for jc in joint_constraints:
            for j in joints:
                penalty_sum += m.joint_penalty[n,j,jc]
        force_sum  += m.F_a[n]**2 + m.Fb_b_total[n,'FF','y_b']**2+m.Fb_b_total[n,'FF','x_b']**2
        torque_sum += m.tau_a[n]**2
    
    return T + force_sum + torque_sum + 10000*penalty_sum
#     return penalty_sum
m.Cost = Objective(rule = CostFun)

In [40]:
#m.pfoot_b.pprint()

In [41]:
# Landing on Board

# INITIAL CONDITIONS

# skateboard
m.pwheel[1,'BW','X'].fix(0.0)
m.pwheel[1,'BW','Y'].fix(0.0)
m.q[1,'th'].fix(0.0)

# monopod
#m.pfoot_b[1,'x_b'].fix(-0.5*m.lbrd)
m.pfoot_b[1,'y_b'].fix(0.0)
m.q[1,'thb'].fix(0.0)
m.q[1,'thl'].fix(0.0)
#m.q[1,'r'].fix(0.25)

# both
for dof in DOFs:
    m.dq[1,dof].fix(0.0)
    
for n in range(1,N+1):
    m.Fb_b[n,'FF','x_b','ps'].fix(0.0)
    m.Fb_b[n,'FF','x_b','ng'].fix(0.0)
    m.Fb_b_total[n,'FF','y_b'].setub(50.0)
#     m.Fb_b_total[n,'FF','y_b'].fix(0.0)
    m.rF[n,'FF'].fix(0.5*m.lbrd)
#     m.F_a[n].fix(0.0)
#     m.tau_a[n].fix(0.0)    
#     m.GRFbf[n,'X'].fix(0.0)

m.ptail[N1-1,'Y'].fix(0.0)

m.q[N1-1,'thb'].fix(0.0)
m.q[N1-1,'thl'].fix(0.0)

Nhw = round(0.5*(N1+N2))
m.q[Nhw,'th'].fix(0.0)
m.pwheel[Nhw,'BW','Y'].setlb(0.3)

m.pwheel[N2,'BW','Y'].fix(0.0)
m.pwheel[N2,'FW','Y'].fix(0.0)
m.q[N2,'thb'].fix(0.0)
m.q[N2,'thl'].fix(0.0)

m.pwheel[N,'BW','Y'].fix(0.0)
m.pwheel[N,'FW','Y'].fix(0.0)
m.q[N,'thb'].fix(0.0)
m.q[N,'thl'].fix(0.0)

#INITIALIZE
if 'm_saved' in globals():
    print("Loading previous model results as initialization.")
    for n in range(1,61):
        for dof in DOFs:
            if not m.q[n,dof].fixed:
                m.q[n,dof] = m_saved.q[n,dof].value
        if not m.F_a[n].fixed:
            m.F_a[n] = m_saved.F_a[n].value
        if not m.tau_a[n].fixed:
            m.tau_a[n] = m_saved.tau_a[n].value
        
        for fs in Fs:
            for dof in DOF_b:
                for sgn in signs:
                    if not m.Fb_b[n,fs,dof,sgn].fixed:
                        m.Fb_b[n,fs,dof,sgn].value
                
        for grf in GRFs:
            if not m.GRF[n, grf].fixed:
                m.GRF[n, grf].value 
print("Done.")

if 'm_saved' not in globals():
    print("New initialization.")
    for n in range(2,N):
        if n != N1 or n != N1-1 or n != Nhw or n != N2:
            m.pfoot_b[n,'x_b'] = np.random.uniform(-0.5*m.lbd,-0.5*m.lbrd)
    eps = 1e-2
    for n in range(1,N+1):
        for gc in ground_constraints:
            m.ground_penalty[n,gc].setub(eps)
        for cc in contact_constraints:
            m.contact_penalty[n,cc].setub(eps)
        for jc in joint_constraints:
            for j in joints:
                m.joint_penalty[n,j,jc].setub(eps)

Loading previous model results as initialization.


In [42]:
# m.pwheel.pprint()

In [43]:
# SOLVE --------------------------------------------------------------------------------------------------------------------  
results = skaterlib.solve(m, display_results = False)
skaterlib.print_results(m, N, ground_constraints, contact_constraints, joint_constraints, joints)

ok
optimal
--------------------------------
GROUND:   1.1955846475340153e-08
CONTACTS: 5.5114132411426854e-08
JOINT:    2.962797333017759e-08
FORCE:    80.08260887331359
TORQUE:   36.35830961330749
--------------------------------


In [44]:
# CALCULATE MAX VALUES
maxvalues = skaterlib.get_max_values(m, N, ground_constraints, contact_constraints, joint_constraints, joints, WDOFs, DOF_b, Fs, signs, GRFs)
maxForce = maxvalues[0]
maxGRF   = maxvalues[1]
maxGRFbf = maxvalues[2]
maxF_a   = maxvalues[3]

In [45]:
# ANIMATE THE RESULTS
from IPython.display import HTML
%matplotlib inline

animate = skaterlib.make_animation(m, N, N1, Nhw, N2, maxForce, maxGRF, maxGRFbf, maxF_a)
HTML(animate.to_html5_video()) #you need to convert the animation to HTML5 to embed it in the notebook

In [46]:
#for i in range(1,N+1):
#    print(i, m.pwheel[i,'BW','Y'].value)

# m.dq.pprint()
# m.contact_penalty.pprint()
# m.pwheel.pprint()

# m.contact_penalty.pprint()

In [47]:
# %reset
#plt.style.available
# print(m.N.pprint())

In [48]:
# import cloudpickle

# with open('model_save_Neq60.pkl', mode='wb') as file:
#     cloudpickle.dump(m, file)

# with open('model_save.pkl', mode='rb') as file:
#     m_saved = cloudpickle.load(file)

# m_saved.q.pprint()
# print("m_saved" in globals())

In [49]:
# ANIMATE THE RESULTS
from IPython.display import HTML
%matplotlib inline

animate_realtime = skaterlib.make_realtime_animation(m, N, hm)
HTML(animate_realtime.to_html5_video()) #you need to convert the animation to HTML5 to embed it in the notebook